In [1]:
!pip install PyMuPDF
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 27.5 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()

Saving Dataset (1).zip to Dataset (1).zip


In [5]:
import zipfile

zip_path = "/content/Dataset (1).zip"
extract_path = "/content/use_cases"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Extracted to:", extract_path)


✅ Extracted to: /content/use_cases


In [2]:
import os
import fitz  # PyMuPDF
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from zipfile import ZipFile
with ZipFile("/content/Dataset (1).zip", 'r') as zip_ref:
    zip_ref.extractall("/content/use_cases")

In [7]:
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Failed to extract {pdf_path}: {e}")
        return ""


In [8]:
def get_all_pdfs(base_dir):
    pdfs = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".pdf"):
                pdfs.append(os.path.join(root, file))
    return pdfs


In [9]:
usecase_5_jd_path = "/content/use_cases/Dataset/[Usecase 5] AI-Powered Job Application Screening System​/job_description.csv"
try:
    jd_df = pd.read_csv(usecase_5_jd_path, encoding='ISO-8859-1')
except:
    jd_df = pd.read_csv(usecase_5_jd_path, encoding='latin1')

jd_text = " ".join(jd_df.iloc[0].dropna().astype(str).values)

resume_dir = "/content/use_cases/Dataset/[Usecase 5] AI-Powered Job Application Screening System​/CVs1"
resume_texts = {}

for file in os.listdir(resume_dir):
    if file.endswith(".pdf"):
        path = os.path.join(resume_dir, file)
        resume_texts[file] = extract_text_from_pdf(path)

In [10]:
conversation_dir = "/content/use_cases/Dataset/[Usecase 7] AI-Driven Customer Support Enhancing Efficiency Through Multiagents​/Conversation"
conversation_texts = {}
for file in os.listdir(conversation_dir):
    if file.endswith(".txt"):
        with open(os.path.join(conversation_dir, file), 'r', encoding='utf-8') as f:
            conversation_texts[file] = f.read()


In [11]:
all_docs = [jd_text] + list(resume_texts.values()) + list(conversation_texts.values())
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(all_docs)


In [12]:
jd_vector = tfidf_matrix[0:1]
resume_vectors = tfidf_matrix[1:1 + len(resume_texts)]
support_vectors = tfidf_matrix[1 + len(resume_texts):]

In [16]:
jd_vector = tfidf_matrix[0:1]
resume_vectors = tfidf_matrix[1:1 + len(resume_texts)]
support_vectors = tfidf_matrix[1 + len(resume_texts):]

resume_similarities = cosine_similarity(jd_vector, resume_vectors).flatten()
support_similarities = cosine_similarity(jd_vector, support_vectors).flatten()


In [17]:
print("\n\n🧾 Resume Screening Results:\n")
THRESHOLD = 80
if 'resume_similarities' in locals():
    for i, resume_name in enumerate(resume_texts.keys()):
        score = round(resume_similarities[i] * 100, 2)
        status = "Shortlisted ✅" if score >= THRESHOLD else "Not Shortlisted ❌"
        print(f"{resume_name}: {score}% match - {status}")
else:
    print("Error: resume_similarities is not defined.")

print("\n\n💬 Customer Support Ticket Match Results:\n")
if 'support_similarities' in locals():
    for i, support_name in enumerate(conversation_texts.keys()):
        score = round(support_similarities[i] * 100, 2)
        print(f"{support_name}: {score}% semantic relevance to job description")
else:
    print("Error: support_similarities is not defined.")




🧾 Resume Screening Results:

C7733.pdf: 6.72% match - Not Shortlisted ❌
C9884.pdf: 4.78% match - Not Shortlisted ❌
C8760.pdf: 3.49% match - Not Shortlisted ❌
C4331.pdf: 3.49% match - Not Shortlisted ❌
C4999.pdf: 3.93% match - Not Shortlisted ❌
C2287.pdf: 4.41% match - Not Shortlisted ❌
C7543.pdf: 6.47% match - Not Shortlisted ❌
C4760.pdf: 3.96% match - Not Shortlisted ❌
C3315.pdf: 4.97% match - Not Shortlisted ❌
C5451.pdf: 5.9% match - Not Shortlisted ❌
C9533.pdf: 3.96% match - Not Shortlisted ❌
C1061.pdf: 3.78% match - Not Shortlisted ❌
C1446.pdf: 5.94% match - Not Shortlisted ❌
C2144.pdf: 4.42% match - Not Shortlisted ❌
C3912.pdf: 5.47% match - Not Shortlisted ❌
C4444.pdf: 4.34% match - Not Shortlisted ❌
C2546.pdf: 6.29% match - Not Shortlisted ❌
C3445.pdf: 3.68% match - Not Shortlisted ❌
C4307.pdf: 5.33% match - Not Shortlisted ❌
C5767.pdf: 5.71% match - Not Shortlisted ❌
C1212.pdf: 6.23% match - Not Shortlisted ❌
C8810.pdf: 5.14% match - Not Shortlisted ❌
C3142.pdf: 3.83% match -